In [1]:
import sys
sys.path.append('../')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

import QUBO_SVM_functions as qSVM
import kernel_SVM_functions as kSVM
import metric_functions as mf
from kernel_SVM_functions import rbf_kernel
from parameter_tuning import tune_csvm_parameters

from quantum_classifier import QSVMq
from quantum_annealing_functions import QA_cross_validate, QA_calibration, stratified_sample

In [2]:
filepath = '../synth_data/synth_0.3.csv'

data = np.loadtxt(filepath, delimiter = ',')
X = data[:, :-1]
t = data[:, -1]

X_train, X_test, t_train, t_test = train_test_split(X, t, train_size = 0.17, shuffle = True, stratify = t)
t_train = t_train.reshape(-1, 1)
t_test = t_test.reshape(-1, 1)
print(X_train.shape)

(20, 2)


In [3]:
B_values = [2, 3]
K_values = [3, 4]
R_values = [0, 1]
gamma_values = [0.125, 0.25, 0.5]

In [9]:
sample_df = pd.read_csv('../QA_results/(2, 3, 1, 2)-f1.txt')
top_models = sample_df.sort_values('energy', ascending = True)[: 20]
encoded_alphas = np.array([list(row[1: -3]) for index, row in top_models.iterrows()])

In [11]:
B = 2
K = 3
N = int(encoded_alphas.shape[0] / K)

b = np.array([B ** i for i in range(K)])
P = np.zeros((N, N * K))
for i in range(N):
    P[i, i * K: (i + 1) * K] = b

In [14]:
encoded_alphas

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.]])

In [13]:
P

array([[1., 2., 4.]])

In [10]:
top_models_arr = np.apply_along_axis(lambda encoded_arr: qSVM.decode(encoded_arr, 2, 3), axis = 1, arr = encoded_alphas)

In [6]:
sample_df

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,29,30,31,32,33,34,35,chain_break_fraction,energy,num_occurrences
0,0,0,0,0,1,1,1,0,1,1,...,1,0,0,0,1,1,1,0.000000,-741.819521,95
1,1,0,0,0,1,1,1,1,1,1,...,1,0,0,0,1,1,1,0.027778,-741.819521,1
2,2,0,0,0,1,1,1,1,1,1,...,1,0,0,0,1,1,1,0.027778,-741.819521,1
3,3,0,0,0,1,1,1,1,1,1,...,1,0,0,0,1,1,1,0.027778,-741.819521,1
4,4,0,0,0,1,1,1,1,1,1,...,1,0,0,0,1,1,1,0.027778,-741.819521,1
5,5,0,0,0,1,1,1,1,1,1,...,1,0,0,0,1,1,1,0.027778,-741.819521,1


In [7]:
encoded_alphas.shape

(6, 36)